<a href="https://colab.research.google.com/github/dunedine/MLDL/blob/main/STS_%EC%9C%A0%EC%82%AC%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers : BERT, GET 계열 

Sentence Transformer (STS 유사도)

In [1]:
!pip install sentence-transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
model_name = "klue/roberta-base"

In [3]:
train_batch_size = 32
num_epochs = 4

In [4]:
from sentence_transformers import SentenceTransformer, losses, models, util

In [5]:
embedding_model =  models.Transformer(model_name)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

In [6]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token = False,
    pooling_mode_max_tokens=False,
)

In [7]:
model = SentenceTransformer(modules=[embedding_model, pooler])

In [8]:
from datasets import load_dataset
datasets = load_dataset("klue", "sts")
datasets
# 해당 datasets  가져온 이유 - fine tuning용
# 이 데이터 labels -> 유사도를 의미

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 11668
    })
    validation: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 519
    })
})

In [9]:
datasets["train"][0] 
# 데이터 중 sentence1,2 가 필요하겠다 판단

{'guid': 'klue-sts-v1_train_00000',
 'source': 'airbnb-rtt',
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

In [10]:
datasets["validation"][0] 

{'guid': 'klue-sts-v1_dev_00000',
 'source': 'airbnb-rtt',
 'sentence1': '무엇보다도 호스트분들이 너무 친절하셨습니다.',
 'sentence2': '무엇보다도, 호스트들은 매우 친절했습니다.',
 'labels': {'label': 4.9, 'real-label': 4.857142857142857, 'binary-label': 1}}

In [11]:
datasets["train"][0]['sentence1']

'숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.'

In [12]:
datasets["train"][0]['sentence2']

'숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.'

In [13]:
datasets["train"][0]['labels']['label']

3.7

In [14]:
from sentence_transformers.readers import InputExample

train_samples = []
for i in range(11668):
  train_samples.append(
      InputExample(
          texts = [datasets["train"][i]['sentence1'], 
                   datasets["train"][i]['sentence2']],
          label = float(datasets["train"][i]['labels']['label'])/5.0,  # /5.0 => 정규화
      )
  )

valid_samples = []
for i in range(519):
  valid_samples.append(
      InputExample(
          texts = [datasets["validation"][i]['sentence1'], 
                   datasets["validation"][i]['sentence2']],
          label = float(datasets["validation"][i]['labels']['label'])/5.0,  # /5.0 => 정규화
      )
  )

In [15]:
train_samples[0].texts, train_samples[0].label

(['숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
  '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.'],
 0.74)

In [16]:
valid_samples[0].texts, valid_samples[0].label

(['무엇보다도 호스트분들이 너무 친절하셨습니다.', '무엇보다도, 호스트들은 매우 친절했습니다.'], 0.9800000000000001)

In [17]:
import torch
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_samples, shuffle=True, batch_size = train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

In [20]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    valid_samples,
    name="sts-valid"
)

In [21]:
import math
warmup_steps = math.ceil(len(train_dataloader)* num_epochs * 0.1)

In [22]:
model_save_path = "/content/drive/MyDrive/Colab Notebooks/동아대2기/my_model"

# Fine tuning

In [24]:
model.fit(
    train_objectives = [(train_dataloader, train_loss)],
    evaluator = evaluator,
    epochs = num_epochs,
    evaluation_steps = 1000,
    warmup_steps = warmup_steps,
    output_path = model_save_path
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

In [27]:
docs = [
    "10일 한국거래소에 따르면 지난 한 달(4월 7일~5월 9일) 동안 외국인 투자자는 코스피 시장에서 3조6176억원어치를 순매수하며 매수 우위를 보였다. 반면 이 기간 개인투자자는 코스피에서 2조9221억원어치를 팔아치웠다. 기관 투자자도 6659억원어치를 순매도했다.",
    "아울러 성남FC 구단주로서 2014년 10월∼2016년 9월 두산건설, 네이버, 차병원, 푸른위례 등 4개 기업의 후원금 133억5천만원을 받는 대가로 건축 인허가나 토지 용도 변경 등 편의를 제공한 혐의도 있다.",
    "피겨 주니어 그랑프리에 데뷔하던 2019년, 14살 중학생 이해인은 쇼팽의 야상곡 20번 올림 다단조를 쇼트프로그램 음악으로 선택해 많은 사람을 놀라게 했다. 보통 사람들에게도 친숙한 쇼팽의 야상곡 2번은 경쾌하고 발랄함을 표현하려는 어린 선수들이 피겨 음악으로 많이 사용해왔지만, 야상곡 20번은 웬만한 선수들이 소화하기 어려운 음악으로 알려져 있기 때문이다.",
    "급기야 끔찍한 살해위협까지 시작되고, 정은 씨의 일상은 한순간에 파괴되고 만다. 한 씨는 다정했던 남자친구가 아니라 공포의 대상으로 변했다. 정은 씨의 집, 직장을 비롯한 모든 것을 알고 있는 사람이 스토커로 돌변하자 그녀에게 안전한 공간은 어디에도 없었다.",
    "르세라핌은 지난 1일 발매된 첫 정규 앨범 '언포기븐'에 대해 '우리는 매번 성장하고 있고 이러한 성장과 긍정적인 변화가 모든 앨범에 담기도록 노력한다. 우리가 전하고 싶은 메시지가 곧 르세라핌이기 때문'이라며 '타인의 시선이나 평가에 개의치 않는 모습이 누군가에게는 빌런처럼 보일 수도 있다. 하지만 우리의 음악이 선을 넘고 경계를 넘어 확장되는 것처럼, 계속해서 한계를 넘어서는 팀이 되고 싶다'고 설명했다."
]

In [28]:
docs_embeddings = model.encode(docs)

In [29]:
query = """세 번째 시리즈로 돌아온 '범죄도시3'는 괴물형사 '마석도'(마동석)가 서울 광역수사대로 이동, 더욱 확장된 세계관을 선보인다. 공개된 캐릭터 스틸만으로도 더 커진 판을 싹 쓸어버릴 마동석표 타격감 넘치는 액션에 기대감을 높인다. 또한 '마석도'만의 유머스러운 모습도 담아내 '범죄도시3'가 선사할 '단짠' 매력을 엿볼 수 있다. 지역도 국경도 제한 없이 나쁜 놈들을 잡기 위해 수사하는 괴물형사 '마석도'의 물 만난 활약이 호기심을 자아낸다."""
query_embedding = model.encode(query)

In [32]:
tok_k = 2
cos_scores = util.pytorch_cos_sim(query_embedding, docs_embeddings)[0]
top_results = torch.topk(cos_scores, k=tok_k)
for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
  print(f"{i}번째 결과: {docs[idx]}, 유사도는 {score} \n")

0번째 결과: 급기야 끔찍한 살해위협까지 시작되고, 정은 씨의 일상은 한순간에 파괴되고 만다. 한 씨는 다정했던 남자친구가 아니라 공포의 대상으로 변했다. 정은 씨의 집, 직장을 비롯한 모든 것을 알고 있는 사람이 스토커로 돌변하자 그녀에게 안전한 공간은 어디에도 없었다., 유사도는 0.21936799585819244 

1번째 결과: 르세라핌은 지난 1일 발매된 첫 정규 앨범 '언포기븐'에 대해 '우리는 매번 성장하고 있고 이러한 성장과 긍정적인 변화가 모든 앨범에 담기도록 노력한다. 우리가 전하고 싶은 메시지가 곧 르세라핌이기 때문'이라며 '타인의 시선이나 평가에 개의치 않는 모습이 누군가에게는 빌런처럼 보일 수도 있다. 하지만 우리의 음악이 선을 넘고 경계를 넘어 확장되는 것처럼, 계속해서 한계를 넘어서는 팀이 되고 싶다'고 설명했다., 유사도는 0.16238945722579956 

